In [1]:
import os
import pandas as pd
import numpy as np
import random
import math
from sklearn.utils import resample

In [2]:
# Change directory 
os.chdir("C:/Kate/Bioinformatics/Diplom")

# Assign spreadsheet filename to `file`
file = 'Selection_434.xlsx'

# Load spreadsheet
xf = pd.ExcelFile(file)

# Print the sheet names
print(xf.sheet_names)

# Load a sheet into a DataFrames by name: dfSelection
dfSelection = xf.parse('Selection')

# Convert names of proteins in dataFrame to list
dataSet = dfSelection["ID"].tolist()

['Sheet1', 'Sheet2', 'Selection', 'List', 'AA', 'tableAA', 'Test']


In [3]:
AA = ['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']
R = sorted(AA)

# Create a DataFrame that will hold the sum of important params from every sample:
dfSum = pd.DataFrame(columns=['Residue','F_RB','F_RE','KBenergy_RB','KBenergy_RE'])
dfSum.Residue = R
dfSum.F_RB = 0
dfSum.F_RE = 0
dfSum.KBenergy_RB = 0
dfSum.KBenergy_RE = 0

In [4]:
# Resample and calculate parameters

path = 'C:/Kate/Bioinformatics/Diplom/Set/'
f = '.xlsx'

repeats = 100 

for n in range(repeats):
    # Change directory 
    os.chdir("C:/Kate/Bioinformatics/Diplom/Set")
    
    # Resample
    # to use random seed 1: random_state=1
    names = resample(dataSet, replace=True, n_samples=20)
    dfNames = pd.DataFrame()
    dfNames['Names'] = names
    
    oob = [x for x in dataSet if x not in names]
    dfoob = pd.DataFrame()
    dfoob['OutOfBox'] = oob
    
    dfWhole = pd.DataFrame(columns=['Index','Residue','sumSASA','rSASA','maxProtOr','RSA','b0e1'])
    
    for i in names:
        #Load a file from the sample 
        protFile = i+f
        # Load spreadsheet
        xf = pd.ExcelFile(protFile)    
        # Load a sheet into a DataFrames by name of the protein
        dfData = xf.parse('data')
    
        # Append the file to form a whole set from the selection files
        dfWhole = dfWhole.append(dfData, ignore_index=True)
        
    # Calculate the Length of the data set
    Length = len(dfWhole)
        
    # Count residue types 
    # R = RE + RB
    numR = dfWhole.groupby('Residue')['b0e1'].count()
        
    # Make sure that the set containes all types of residues
    if len(numR) < 20:
        break
        
    dfParams = pd.DataFrame()
    dfParams['Residue'] = numR.index
    dfParams['numR'] = numR.values
               
    # Count the buried residues per type
    subNumRB = dfWhole[dfWhole['b0e1'] == 0].groupby('Residue')['b0e1'].count()
    # Count the buried residues per type
    subNumRE = dfWhole[dfWhole['b0e1'] == 1].groupby('Residue')['b0e1'].count()
        
    R = dfParams.Residue.values
    numRB = []
    numRE = []
    for r in R:
        if r in subNumRB.index:
            numRB.append(subNumRB.at[r])
        else:
            numRB.append(0)
                
        if r in subNumRE.index:
            numRE.append(subNumRE.at[r])
        else:
            numRE.append(0)
        
    dfParams['RB'] = numRB
    dfParams['RE'] = numRE
    
    dfParams['F_RB'] = dfParams.RB/Length
    dfParams['F_RE'] = dfParams.RE/Length
    
    # Count buried and exposed residues
    numBE = dfWhole.groupby('b0e1')['Residue'].count()
        
    # Calculate the expected probability for a residue to be Buried 
    expPB = numBE.at[0]/Length
    # Calculate the expected probability for a residue to be Exposed 
    expPE = numBE.at[1]/Length
        
    # Calculate the oberved probability that a residue of certain type is buried 
    dfParams['PobsRB'] = dfParams.RB/numR.values
    # Calculate the oberved probability that a residue of certain type is buried
    dfParams['PobsRE'] = dfParams.RE/numR.values
        
    # LOG base 10 - 'log10'
    KBenergyRB_log10 = []
    for p in dfParams.PobsRB.values:
        KBenergyRB_log10.append(math.log10(expPB/p))           
    # LOG base 10 - 'log10'
    KBenergyRE_log10 = []
    for p in dfParams.PobsRE.values:
        KBenergyRE_log10.append(math.log10(expPE/p))
        
    # LOG with base 'e' (ln) !!  
    KBenergyRB = []
    for p in dfParams.PobsRB.values:
        KBenergyRB.append(math.log(expPB/p))
    # LOG with base 'e' (ln) !!  
    KBenergyRE = []
    for p in dfParams.PobsRE.values:
         KBenergyRE.append(math.log(expPE/p))
        
    # Save the rest of all the parameters in a DataFrame dfParams
    dfParams['KBenergy_RB'] = KBenergyRB
    dfParams['KBenergy_RE'] = KBenergyRE
    dfParams['KBenergyRB_log10'] = KBenergyRB_log10
    dfParams['KBenergyRE_log10'] = KBenergyRE_log10
    
    # Save the parameters for Entropy and Energy calc in a DataFrame dfPKBES
    dfKBES = pd.DataFrame()
    dfKBES['Residue'] = numR.index
    dfKBES['F_RB'] = dfParams.F_RB
    dfKBES['F_RE'] = dfParams.F_RE
    dfKBES['KBenergy_RB'] = KBenergyRB
    dfKBES['KBenergy_RE'] = KBenergyRE
    
    # Add values to the DataFrame that holds the sum of important params:
    dfSum.F_RB = dfSum.F_RB + dfKBES.F_RB
    dfSum.F_RE = dfSum.F_RE + dfKBES.F_RE
    dfSum.KBenergy_RB = dfSum.KBenergy_RB + dfKBES.KBenergy_RB
    dfSum.KBenergy_RE = dfSum.KBenergy_RE + dfKBES.KBenergy_RE
        
    # Change directory 
    os.chdir("C:/Kate/Bioinformatics/Diplom/Resample20x100/")
    writer = pd.ExcelWriter(('Sample' + str(n) + '.xlsx'), engine='xlsxwriter')

    # Write your DataFrame to a file     
    dfNames.to_excel(writer, 'Sample')
    dfKBES.to_excel(writer, 'KBES')
    dfParams.to_excel(writer, 'Params')
    numBE.to_excel(writer, 'numBE')
    dfWhole.to_excel(writer, 'FullSet')
    dfoob.to_excel(writer, 'OutOfBox')

    # Save the result 
    writer.save()    

In [5]:
dfSum

,Residue,F_RB,F_RE,KBenergy_RB,KBenergy_RE
0,A,3.762349,4.324526,-30.403715,20.689865
1,C,0.964540,0.650312,-55.374660,50.425430
2,D,0.833446,5.003087,88.506120,-26.526781
3,E,0.601553,5.918592,133.731006,-32.259543
4,F,2.278479,1.599381,-53.906728,47.208016
5,G,2.471134,4.709723,0.384346,0.154937
6,H,0.626356,1.871540,33.529679,-13.014239
7,I,3.634416,2.069541,-62.035078,59.725369
8,K,0.270567,5.846585,210.916204,-37.421845
9,L,5.444914,3.672671,-55.440832,49.080971


In [6]:
dfAverage = pd.DataFrame()
dfAverage['Residue'] = dfSum.Residue
dfAverage['F_RB'] = dfSum.F_RB/repeats
dfAverage['F_RE'] = dfSum.F_RE/repeats
dfAverage['KBenergy_RB'] = dfSum.KBenergy_RB/repeats
dfAverage['KBenergy_RE'] = dfSum.KBenergy_RE/repeats
dfAverage

,Residue,F_RB,F_RE,KBenergy_RB,KBenergy_RE
0,A,0.037623,0.043245,-0.304037,0.206899
1,C,0.009645,0.006503,-0.553747,0.504254
2,D,0.008334,0.050031,0.885061,-0.265268
3,E,0.006016,0.059186,1.337310,-0.322595
4,F,0.022785,0.015994,-0.539067,0.472080
5,G,0.024711,0.047097,0.003843,0.001549
6,H,0.006264,0.018715,0.335297,-0.130142
7,I,0.036344,0.020695,-0.620351,0.597254
8,K,0.002706,0.058466,2.109162,-0.374218
9,L,0.054449,0.036727,-0.554408,0.490810


In [7]:
# Change directory 
os.chdir("C:/Kate/Bioinformatics/Diplom/Set/")
writer = pd.ExcelWriter(('Params_size20_repeats' + str(repeats) + '.xlsx'), engine='xlsxwriter')

# Write your DataFrame to a file     
dfSum.to_excel(writer, 'Total')
dfAverage.to_excel(writer, 'KBES')

# Save the result 
writer.save()    